# SwellSight Pipeline - Enhanced Data Augmentation System

This enhanced notebook implements FLUX.1-dev optimized augmentation parameter generation with improved validation, memory optimization, and progress tracking.

## Key Improvements
- **FLUX.1-dev Optimization**: Parameter ranges and distributions optimized for FLUX.1-dev model
- **Enhanced Validation**: Comprehensive parameter validation with quality checks
- **Memory Optimization**: Dynamic memory monitoring and efficient parameter generation
- **Progress Tracking**: Real-time progress with memory usage display
- **Configuration Snapshots**: Reproducibility through configuration preservation
- **Parameter Diversity Analysis**: Advanced diversity metrics and reporting

## Pipeline Integration
This notebook implements Step 4 of the enhanced pipeline:
1. **Load Configuration**: Enhanced config loading with hardware adaptation
2. **Parameter System**: FLUX-optimized parameter generator with validation
3. **Batch Generation**: Memory-efficient parameter set creation
4. **Quality Control**: Advanced parameter validation and quality checks
5. **Diversity Analysis**: Comprehensive parameter space coverage analysis
6. **Export & Snapshot**: Results export with configuration preservation

---

## 1. Enhanced Configuration and Setup

In [ ]:
import sysimport jsonimport loggingfrom pathlib import Pathfrom datetime import datetimeimport numpy as npimport matplotlib.pyplot as pltimport seaborn as snsfrom tqdm.auto import tqdmimport randomfrom collections import defaultdictimport warningsimport gcimport psutilwarnings.filterwarnings('ignore')# Add src to path for production module importssrc_path = Path.cwd() / "src"if str(src_path) not in sys.path:    sys.path.insert(0, str(src_path))# Import production modulestry:    from swellsight.utils.config import load_config, validate_config    from swellsight.utils.error_handler import ErrorHandler, retry_with_backoff    from swellsight.utils.hardware import HardwareManager    from swellsight.utils.performance import PerformanceOptimizer, OptimizationConfig    from swellsight.utils.monitoring import SystemMonitor    from swellsight.data.augmentation import WaveAugmentation, AugmentationConfig    from swellsight.evaluation.data_quality import DataQualityAssessor        print("✅ Production modules loaded")    PRODUCTION_MODULES_AVAILABLE = True        # Initialize production utilities    error_handler = ErrorHandler()    hardware_manager = HardwareManager()    perf_optimizer = PerformanceOptimizer()    system_monitor = SystemMonitor()    data_quality_assessor = DataQualityAssessor()    except ImportError as e:    print(f"⚠️  Production modules unavailable: {e}")    print("   Some enhanced features may not be available")    PRODUCTION_MODULES_AVAILABLE = False    error_handler = hardware_manager = perf_optimizer = None    system_monitor = data_quality_assessor = None# Configure enhanced logginglogging.basicConfig(    level=logging.INFO,    format='%(asctime)s - %(levelname)s - %(message)s')logger = logging.getLogger(__name__)print("🔄 Loading enhanced configuration and utilities...")print("✅ Enhanced utilities initialized successfully")

In [ ]:
# Load enhanced configuration with hardware adaptation
print("🔧 Loading enhanced configuration...")

try:
    # Load configuration with hardware detection and adaptation
    PIPELINE_CONFIG = config_manager.load_config()
    
    # Validate configuration
    validation_result = validate_config(PIPELINE_CONFIG)
    if not validation_result['valid']:
        logger.warning("Configuration validation issues found:")
        for error in validation_result['errors']:
            logger.warning(f"  - {error}")
    
    if validation_result['warnings']:
        logger.info("Configuration warnings:")
        for warning in validation_result['warnings']:
            logger.info(f"  - {warning}")
    
    # Load previous stage results
    depth_results = data_flow_manager.load_previous_results(
        stage_name="depth_extraction",
        required_files=["depth_maps.json", "depth_quality.json"]
    )
    
    print("✅ Configuration and previous results loaded successfully")
    
    # Display configuration summary
    print(f"\n📋 Enhanced Pipeline Configuration:")
    print(f"   Pipeline: {PIPELINE_CONFIG['pipeline']['name']} v{PIPELINE_CONFIG['pipeline']['version']}")
    print(f"   Batch size: {PIPELINE_CONFIG['processing']['batch_size']}")
    print(f"   Quality threshold: {PIPELINE_CONFIG['processing']['quality_threshold']}")
    print(f"   Memory limit: {PIPELINE_CONFIG['processing']['memory_limit_gb']:.1f}GB")
    
    # Display hardware info if available
    if 'detected_hardware' in PIPELINE_CONFIG:
        hw_info = PIPELINE_CONFIG['detected_hardware']
        print(f"\n🖥️  Detected Hardware:")
        print(f"   GPU Available: {hw_info['gpu_available']}")
        if hw_info['gpu_available']:
            print(f"   GPU: {hw_info['gpu_name']}")
            print(f"   GPU Memory: {hw_info['gpu_memory_gb']:.1f}GB")
            print(f"   FLUX Compatibility: {hw_info['flux_compatibility']}")
        print(f"   CPU Cores: {hw_info['cpu_count']}")
        print(f"   System Memory: {hw_info['memory_gb']:.1f}GB")
    
    # Display depth extraction results
    if depth_results:
        print(f"\n📊 Depth Extraction Results:")
        print(f"   Images processed: {len(depth_results.get('images', []))}")
        print(f"   Average quality: {depth_results.get('average_quality', 0):.3f}")
        print(f"   Ready for augmentation: {len(depth_results.get('images', []))} images")
    
except Exception as e:
    logger.error(f"Error loading configuration: {e}")
    print("❌ Failed to load configuration. Please check previous pipeline stages.")
    sys.exit(1)

## 2. FLUX-Optimized Parameter System

In [ ]:
class FLUXOptimizedParameterSystem:
    """
    Enhanced parameter system optimized for FLUX.1-dev model
    with comprehensive validation and quality checks
    """
    
    def __init__(self, config: dict):
        self.config = config
        self.flux_config = config['models']['base_model']
        self.controlnet_config = config['models']['controlnet_model']
        
        # FLUX.1-dev optimized parameter ranges
        self.parameter_ranges = {
            'guidance_scale': {
                'min': 1.0,
                'max': 5.0,  # FLUX works better with lower guidance
                'default': 3.5,
                'distribution': 'normal',
                'std': 0.8
            },
            'num_inference_steps': {
                'values': [20, 25, 28, 30, 35],  # FLUX optimal step counts
                'default': 28,
                'weights': [0.1, 0.2, 0.4, 0.2, 0.1]  # Prefer 28 steps
            },
            'controlnet_conditioning_scale': {
                'min': 0.4,
                'max': 0.8,  # FLUX ControlNet works better with moderate conditioning
                'default': 0.6,
                'distribution': 'uniform'
            },
            'control_guidance_start': {
                'min': 0.0,
                'max': 0.2,
                'default': 0.0,
                'distribution': 'uniform'
            },
            'control_guidance_end': {
                'min': 0.8,
                'max': 1.0,
                'default': 1.0,
                'distribution': 'uniform'
            }
        }
        
        # Enhanced prompt variations for wave/ocean scenes
        self.prompt_variations = [
            "ocean waves crashing on beach, natural lighting, photorealistic",
            "powerful sea waves, dramatic coastline, high detail",
            "coastal waves breaking, golden hour lighting, cinematic",
            "surf waves rolling to shore, clear blue water, realistic",
            "ocean swells approaching beach, natural colors, detailed",
            "breaking waves with white foam, sunny day, sharp focus",
            "sea waves in motion, dynamic water, professional photography",
            "beach waves at sunset, warm lighting, high resolution",
            "turbulent ocean waves, stormy atmosphere, dramatic",
            "gentle waves lapping shore, peaceful scene, soft lighting"
        ]
        
        # Parameter validation rules
        self.validation_rules = {
            'guidance_scale': {
                'min_value': 0.5,
                'max_value': 10.0,
                'recommended_min': 1.0,
                'recommended_max': 5.0
            },
            'num_inference_steps': {
                'min_value': 10,
                'max_value': 50,
                'recommended_min': 20,
                'recommended_max': 35
            },
            'controlnet_conditioning_scale': {
                'min_value': 0.1,
                'max_value': 2.0,
                'recommended_min': 0.4,
                'recommended_max': 0.8
            }
        }
    
    def generate_random_parameters(self) -> dict:
        """Generate FLUX-optimized random parameters"""
        try:
            parameters = {}
            
            # Guidance scale (normal distribution around optimal value)
            guidance_config = self.parameter_ranges['guidance_scale']
            if guidance_config['distribution'] == 'normal':
                guidance = np.random.normal(guidance_config['default'], guidance_config['std'])
                guidance = np.clip(guidance, guidance_config['min'], guidance_config['max'])
            else:
                guidance = np.random.uniform(guidance_config['min'], guidance_config['max'])
            parameters['guidance_scale'] = float(guidance)
            
            # Inference steps (weighted choice)
            steps_config = self.parameter_ranges['num_inference_steps']
            steps = np.random.choice(steps_config['values'], p=steps_config['weights'])
            parameters['num_inference_steps'] = int(steps)
            
            # ControlNet conditioning scale
            conditioning_config = self.parameter_ranges['controlnet_conditioning_scale']
            conditioning = np.random.uniform(conditioning_config['min'], conditioning_config['max'])
            parameters['controlnet_conditioning_scale'] = float(conditioning)
            
            # Control guidance timing
            start_config = self.parameter_ranges['control_guidance_start']
            end_config = self.parameter_ranges['control_guidance_end']
            parameters['control_guidance_start'] = float(np.random.uniform(start_config['min'], start_config['max']))
            parameters['control_guidance_end'] = float(np.random.uniform(end_config['min'], end_config['max']))
            
            # Random seed for reproducibility
            parameters['seed'] = int(np.random.randint(0, 2**32 - 1))
            
            # Prompt variation
            parameters['prompt_variation'] = np.random.choice(self.prompt_variations)
            
            return parameters
            
        except Exception as e:
            logger.error(f"Error generating parameters: {e}")
            return self._get_default_parameters()
    
    def validate_parameters(self, parameters: dict) -> dict:
        """Validate parameter values and provide quality assessment"""
        validation_result = {
            'valid': True,
            'issues': [],
            'warnings': [],
            'quality_score': 1.0
        }
        
        try:
            quality_scores = []
            
            for param_name, value in parameters.items():
                if param_name in self.validation_rules:
                    rules = self.validation_rules[param_name]
                    
                    # Check absolute bounds
                    if value < rules['min_value'] or value > rules['max_value']:
                        validation_result['valid'] = False
                        validation_result['issues'].append(
                            f"{param_name} value {value} outside valid range [{rules['min_value']}, {rules['max_value']}]"
                        )
                        quality_scores.append(0.0)
                        continue
                    
                    # Check recommended bounds
                    if value < rules['recommended_min'] or value > rules['recommended_max']:
                        validation_result['warnings'].append(
                            f"{param_name} value {value} outside recommended range [{rules['recommended_min']}, {rules['recommended_max']}]"
                        )
                        quality_scores.append(0.7)  # Reduced quality but still valid
                    else:
                        quality_scores.append(1.0)  # Optimal quality
            
            # Calculate overall quality score
            if quality_scores:
                validation_result['quality_score'] = np.mean(quality_scores)
            
            return validation_result
            
        except Exception as e:
            logger.error(f"Error validating parameters: {e}")
            return {
                'valid': False,
                'issues': [f"Validation error: {str(e)}"],
                'warnings': [],
                'quality_score': 0.0
            }
    
    def _get_default_parameters(self) -> dict:
        """Get default parameters as fallback"""
        return {
            'guidance_scale': self.parameter_ranges['guidance_scale']['default'],
            'num_inference_steps': self.parameter_ranges['num_inference_steps']['default'],
            'controlnet_conditioning_scale': self.parameter_ranges['controlnet_conditioning_scale']['default'],
            'control_guidance_start': self.parameter_ranges['control_guidance_start']['default'],
            'control_guidance_end': self.parameter_ranges['control_guidance_end']['default'],
            'seed': 42,
            'prompt_variation': self.prompt_variations[0]
        }

# Initialize FLUX-optimized parameter system
print("🎛️  Initializing FLUX-optimized parameter system...")

try:
    param_system = FLUXOptimizedParameterSystem(PIPELINE_CONFIG)
    
    # Test parameter generation
    test_params = param_system.generate_random_parameters()
    validation_result = param_system.validate_parameters(test_params)
    
    print("✅ FLUX-optimized parameter system initialized successfully!")
    print(f"   Parameter validation: {'✅ Valid' if validation_result['valid'] else '❌ Invalid'}")
    print(f"   Quality score: {validation_result['quality_score']:.3f}")
    print(f"   Sample parameters: {test_params}")
    
    if validation_result['warnings']:
        print(f"   Warnings: {len(validation_result['warnings'])}")
    
except Exception as e:
    logger.error(f"Failed to initialize parameter system: {e}")
    print("❌ Parameter system initialization failed")
    sys.exit(1)

## 3. Memory-Optimized Parameter Generation

In [ ]:
# Memory-optimized batch parameter generation
print("🎲 Starting memory-optimized parameter generation...")

# Get images ready for augmentation
ready_images = depth_results.get('images', [])
if not ready_images:
    print("❌ No images ready for augmentation. Please check previous pipeline stages.")
    sys.exit(1)

# Get synthetic per real from config
synthetic_per_real = PIPELINE_CONFIG['processing'].get('synthetic_per_real', 3)
total_parameter_sets = len(ready_images) * synthetic_per_real

print(f"\n📊 Generation Configuration:")
print(f"   Images to process: {len(ready_images)}")
print(f"   Synthetic variations per image: {synthetic_per_real}")
print(f"   Total parameter sets to generate: {total_parameter_sets}")

# Monitor memory usage
initial_memory = monitor_memory()
print(f"\n💾 Initial Memory Usage:")
print(f"   System: {initial_memory.get('system_percent', 0):.1f}%")
if 'gpu_percent' in initial_memory:
    print(f"   GPU: {initial_memory.get('gpu_percent', 0):.1f}%")

# Calculate optimal batch size for parameter generation
param_memory_per_item = 1024  # Estimated bytes per parameter set
optimal_batch_size = memory_optimizer.get_optimal_batch_size(
    item_size=param_memory_per_item,
    max_batch_size=min(len(ready_images), 100)
)

print(f"\n🔧 Memory Optimization:")
print(f"   Optimal batch size: {optimal_batch_size}")
print(f"   Estimated memory per parameter set: {param_memory_per_item} bytes")

# Initialize tracking variables
all_augmentation_params = []
parameter_statistics = defaultdict(list)
generation_metadata = []
validation_summary = {
    'total_generated': 0,
    'valid_parameters': 0,
    'invalid_parameters': 0,
    'quality_warnings': 0,
    'average_quality_score': 0.0
}

print(f"\n🚀 Starting parameter generation with memory monitoring...")

# Use memory monitoring context
with memory_optimizer.memory_monitor(log_usage=True) as monitor:
    # Process images in batches for memory efficiency
    for batch_start in tqdm(range(0, len(ready_images), optimal_batch_size), 
                           desc="Processing batches", unit="batch"):
        
        batch_end = min(batch_start + optimal_batch_size, len(ready_images))
        batch_images = ready_images[batch_start:batch_end]
        
        # Generate parameters for current batch
        batch_params = []
        
        for i, image_path in enumerate(batch_images):
            image_path_obj = Path(image_path)
            global_index = batch_start + i
            
            # Generate multiple parameter sets for this image
            for j in range(synthetic_per_real):
                try:
                    # Generate parameters
                    augmentation_params = param_system.generate_random_parameters()
                    
                    # Validate parameters
                    validation_result = param_system.validate_parameters(augmentation_params)
                    
                    # Create parameter metadata
                    param_metadata = {
                        'image_path': image_path,
                        'image_name': image_path_obj.name,
                        'variation_index': j + 1,
                        'total_variations': synthetic_per_real,
                        'global_index': validation_summary['total_generated'],
                        'parameters': augmentation_params,
                        'validation': validation_result,
                        'generated_timestamp': datetime.now().isoformat()
                    }
                    
                    batch_params.append(param_metadata)
                    
                    # Update validation summary
                    validation_summary['total_generated'] += 1
                    if validation_result['valid']:
                        validation_summary['valid_parameters'] += 1
                    else:
                        validation_summary['invalid_parameters'] += 1
                    
                    validation_summary['quality_warnings'] += len(validation_result['warnings'])
                    
                    # Collect statistics for numeric parameters
                    for key, value in augmentation_params.items():
                        if isinstance(value, (int, float)):
                            parameter_statistics[key].append(value)
                    
                except Exception as e:
                    logger.error(f"Failed to generate parameters for {image_path_obj.name} variation {j+1}: {e}")
                    validation_summary['invalid_parameters'] += 1
                    continue
        
        # Add batch to main collection
        all_augmentation_params.extend(batch_params)
        
        # Monitor memory usage during processing
        current_memory = monitor_memory()
        if current_memory.get('system_percent', 0) > 85:
            logger.warning(f"High memory usage detected: {current_memory.get('system_percent', 0):.1f}%")
            # Force garbage collection
            gc.collect()
        
        # Clear batch variables to free memory
        del batch_params
        gc.collect()

# Calculate final statistics
if validation_summary['total_generated'] > 0:
    # Calculate average quality score
    quality_scores = [param['validation']['quality_score'] for param in all_augmentation_params 
                     if 'validation' in param]
    if quality_scores:
        validation_summary['average_quality_score'] = np.mean(quality_scores)

print(f"\n✅ Parameter generation completed!")
print(f"\n📊 Generation Results:")
print(f"   Total parameter sets generated: {validation_summary['total_generated']}")
print(f"   Valid parameter sets: {validation_summary['valid_parameters']}")
print(f"   Invalid parameter sets: {validation_summary['invalid_parameters']}")
print(f"   Quality warnings: {validation_summary['quality_warnings']}")
print(f"   Average quality score: {validation_summary['average_quality_score']:.3f}")
print(f"   Success rate: {validation_summary['valid_parameters']/validation_summary['total_generated']*100:.1f}%")

# Final memory check
final_memory = monitor_memory()
print(f"\n💾 Final Memory Usage:")
print(f"   System: {final_memory.get('system_percent', 0):.1f}%")
if 'gpu_percent' in final_memory:
    print(f"   GPU: {final_memory.get('gpu_percent', 0):.1f}%")